In [27]:
from world import *

In [67]:
socket = world_socket("vcm-32083.vm.duke.edu", 23456)

In [68]:
wh0 = create_warehouse(0, 0, 0)
msg = connect_new(socket, [wh0])
print(msg)

worldid: 8
result: "connected!"



In [69]:
T800 = create_product(0, "T-900", 20)
T800_purchase = create_purchase(0, [T800], 0)
msg = world_amazon_pb2.ACommands()
msg.buy.append(T800_purchase)
msg.disconnect = False
retmsg = send_message(socket, msg)

In [70]:

aresponse_msg = world_amazon_pb2.AResponses()
aresponse_msg.ParseFromString(retmsg)
print(aresponse_msg)


arrived {
  whnum: 0
  things {
    id: 0
    description: "T-900"
    count: 20
  }
  seqnum: 10
}
acks: 0



In [71]:
send_ack(socket, [0])

In [76]:
T800 = create_product(0, "T-900", 20)
T800_pack = create_pack(0, [T800], 0, 2)
msg = world_amazon_pb2.ACommands()
msg.topack.append(T800_pack)
msg.disconnect = False
retmsg = send_message(socket, msg)

aresponse_msg = world_amazon_pb2.AResponses()
aresponse_msg.ParseFromString(retmsg)
print(aresponse_msg)

acks: 2



In [78]:
send_ack(socket, [2])

In [1]:

import sqlalchemy

from datetime import datetime

from sqlalchemy import Table, create_engine, MetaData, text, Column, Integer, String, Float, Sequence, ForeignKey, inspect
from sqlalchemy import Enum, Float, PrimaryKeyConstraint, DateTime, func, UniqueConstraint
from sqlalchemy.engine import URL
from sqlalchemy.orm import relationship, declarative_base, subqueryload, sessionmaker

engine = create_engine(URL.create(username = 'amazon',
                                  password = 'AlchOfRevo12A2',
                                  database = 'amazon',
                                  drivername = 'postgresql'))

Base = declarative_base()

class product(Base):
    __tablename__ = "product"
    
    id = Column(Integer, Sequence("product_id_seq", start=1,increment=1), primary_key = True)
    descrption = Column(String)
    
class warehouse(Base):
    __tablename__ = "warehouse"

    id = Column(Integer, Sequence("warehouse_id_seq", start=1,increment=1), primary_key = True)
    x = Column(Integer)
    y = Column(Integer)

class inventory(Base):
    __tablename__ = "inventory"

    id = Column(Integer, Sequence("inventory_id_seq", start=1,increment=1), primary_key = True)
    pid = Column(Integer, ForeignKey('product.id'))
    wid = Column(Integer, ForeignKey('warehouse.id'))
    amount = Column(Integer)

    isProduct = relationship("product", backref="inventory")
    storedIn = relationship("warehouse", backref="inventory")

    __table_args__ = (UniqueConstraint(pid, wid),)

class order(Base):
    __tablename__ = "order"

    id = Column(Integer, Sequence("order_id_seq", start=1,increment=1), primary_key = True)
    pid = Column(Integer, ForeignKey('product.id'))
    wid = Column(Integer, ForeignKey('warehouse.id'))
    tid = Column(Integer)
    uid = Column(Integer)
    status = Column(Enum("created", "packed", "shipped", "delivered", "canceled", name='status', create_type=False), nullable=False)
    
    isProduct = relationship("product", backref="order")
    packedFrom = relationship("warehouse", backref="order")

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine, checkfirst=True)

In [10]:

import db
init_db()


ImportError: attempted relative import with no known parent package

In [2]:
db_add_product("T800")
db_add_product("ppsh41")

In [3]:
products = query_products()
for p in products:

    print( p.descrption)

T800
ppsh41


In [4]:
socket = world_socket("vcm-32083.vm.duke.edu", 23456)
warehouse0 = create_warehouse(0, 0, 0)
add_warehouse(0, 0, 0)
connect_new(socket, [warehouse0])

worldid: 4
result: "connected!"

In [5]:
db_add_inventory("T800", 0, 500, socket)
db_add_inventory("ppsh41", 0, 500, socket)

T800
arrived {
  whnum: 0
  things {
    id: 1
    description: "T800"
    count: 500
  }
  seqnum: 5
}
acks: 0

ppsh41
arrived {
  whnum: 0
  things {
    id: 2
    description: "ppsh41"
    count: 500
  }
  seqnum: 6
}
acks: 1



In [7]:
inv = query_inventory()
for p in inv:
    print(p.product.descrption, p.warehouse.wid, p.warehouse.x, p.warehouse.y, p.inventory.amount)


T800 0 0 0 200
ppsh41 0 0 0 200
